In [1]:
from pathlib import Path
from omop_semantics.runtime import OmopSemanticEngine, RegistryFragment
from omop_semantics.runtime.instance_loader import load_registry_fragment, merge_instance_files, load_profiles, load_symbol_module
from linkml_runtime.loaders import yaml_loader
from omop_semantics import SCHEMA_DIR, INSTANCE_DIR

instance_base = INSTANCE_DIR
profile_base = SCHEMA_DIR / "profiles"

# we don't want to be either
# 1. redefining full classes for simple data structures that 
#    conform exactly to the defined profiles, 
# or 
# 2. having to repeat full profiles in registry fragments. Instead, we
#    have a pre-processing step that merges instance files with profiles
#    to create a "registry fragment" that can be used to initialise the resolver

profiles = load_profiles(instance_base / "profiles.yaml")

registry_dict = merge_instance_files(
    paths = [instance_base / "demographic.yaml"],
    profiles = profiles
)

In [2]:
# note how on initial load, we just get a reference to the template as a string, which does not
# conform to the template structure in its own right
yaml_loader.load_as_dict(f'{instance_base / "demographic.yaml"}')['groups'][0]['registry_members'][0]

{'name': 'Country of birth',
 'role': 'demographic',
 'entity_concept': {'name': 'Country of Birth',
  'class_uri': 'OmopGroup',
  'parent_concepts': [{'concept_id': 4155450, 'label': 'Country of birth'}]},
 'cdm_profile': 'observation_simple'}

In [3]:
# after we apply the profiles, we get a fully conformed registry 
# fragment with all the relevant information from the profile
profiles['observation_simple']

OmopCdmProfile(name='observation_simple', cdm_table='observation', concept_slot='observation_concept_id', value_slot=None)

In [4]:
# this structure can now be used to properly instantiate the template classes
# and then fully used for rendering, validation, etc.
registry_dict['groups'][0]['registry_members'][0]

{'name': 'Country of birth',
 'role': 'demographic',
 'entity_concept': {'name': 'Country of Birth',
  'class_uri': 'OmopGroup',
  'parent_concepts': [{'concept_id': 4155450, 'label': 'Country of birth'}]},
 'cdm_profile': {'name': 'observation_simple',
  'cdm_table': 'observation',
  'concept_slot': 'observation_concept_id',
  'value_slot': None}}

In [5]:
# other more complex modules can be created as full linkml subschemas
# and then loaded directly instead
staging_symbols = load_symbol_module(profile_base / 'omop_staging.yaml')
modifier_symbols = load_symbol_module(profile_base / 'omop_modifiers.yaml')

In [6]:
staging_symbols['TStageConcepts']

{'is_a': 'RegistryGroup',
 'name': 'TStageConcepts',
 'role': 'staging',
 'registry_members': ['T0', 'T1', 'T2', 'T3', 'T4', 'Ta', 'Tis', 'TX']}

In [7]:
staging_symbols['T0']

{'class_uri': 'OmopGroup',
 'parent_concepts': [{'concept_id': 1634213, 'label': 'T0'}],
 'role': 'staging'}

In [8]:
registry_fragment = yaml_loader.load(
    registry_dict,
    target_class=RegistryFragment
)

In [9]:
engine = OmopSemanticEngine(
    registry_fragment=registry_fragment,
    profile_objects=[staging_symbols, modifier_symbols]   # or merge with profile symbols too
)

In [10]:
engine.registry_runtime.get_runtime('Country of birth')

RuntimeTemplate(name='Country of birth', role='demographic', cdm_profile=OmopCdmProfile(name='observation_simple', cdm_table='observation', concept_slot='observation_concept_id', value_slot=None), entity_concept_ids={4155450}, value_concept_ids=None)

In [11]:
engine.registry_runtime.roles

{'demographic'}

In [12]:
engine.registry_runtime.template_names

{'Country of birth', 'Language spoken', 'Postcode'}

In [15]:
engine

In [27]:
import old_enumerators
import enum

In [ ]:
import old_enumerators
import enum

target_vals = {}
for k, v in vars(old_enumerators).items():
    if type(v) == enum.EnumType:
        target_vals[k] = {lab: val for val, lab in zip(v.member_values(), v.labels())}

In [36]:
all_vals = [x for y in target_vals.values() for x in y.values()]

In [39]:
target_vals

{'ConceptEnum': {},
 'Unknown': {'generic': 4129922,
  'gender': 4214687,
  'condition': 44790729,
  'cancer': 36402660,
  'grade': 4264626,
  'stage': 36768646,
  'stage_edition': 1634449,
  'cob': 40482029,
  'drug_trial': 4090378,
  'therapeutic_regimen': 4207655},
 'ModifierFields': {'condition_occurrence_id': 1147127,
  'drug_exposure_id': 1147707,
  'procedure_occurrence_id': 1147082,
  'episode_id': 756290},
 'ModifierTables': {'drug_exposure': 1147339,
  'episode': 35225440,
  'observation': 1147304},
 'TreatmentEpisode': {'treatment_regimen': 32531,
  'treatment_cycle': 32532,
  'cancer_surgery': 32939,
  'radiotherapy': 32940},
 'Modality': {'chemotherapy': 35803401, 'radiotherapy': 35803411},
 'DiseaseEpisodeConcepts': {'episode_of_care': 32533,
  'confined': 32528,
  'invasive': 32677,
  'metastatic': 32944,
  'stable_disease': 32948,
  'disease_progression': 32949,
  'partial_response': 32947},
 'EpisodeTypes': {'ehr_defined': 32544,
  'ehr_derived': 32545,
  'ehr_prescrip

In [38]:
engine.profile_runtime.objects

{'StagingAxis': {'is_a': 'RegistryFragment',
  'name': 'StagingAxis',
  'role': 'staging',
  'groups': ['TStageConcepts',
   'NStageConcepts',
   'MStageConcepts',
   'GroupStageConcepts'],
  'notes': 'Staging axes for neoplastic disease'},
 'TStageConcepts': {'is_a': 'RegistryGroup',
  'name': 'TStageConcepts',
  'role': 'staging',
  'registry_members': ['T0', 'T1', 'T2', 'T3', 'T4', 'Ta', 'Tis', 'TX']},
 'NStageConcepts': {'is_a': 'RegistryGroup',
  'name': 'NStageConcepts',
  'role': 'staging',
  'registry_members': ['N0', 'N1', 'N2', 'N3', 'N4', 'NX']},
 'MStageConcepts': {'is_a': 'RegistryGroup',
  'name': 'MStageConcepts',
  'role': 'staging',
  'registry_members': ['M0', 'M1', 'MX']},
 'GroupStageConcepts': {'is_a': 'RegistryGroup',
  'name': 'GroupStageConcepts',
  'role': 'staging',
  'registry_members': ['group_stage']},
 'group_stage': {'is_a': 'OmopTemplate',
  'name': 'Group Stage',
  'entity_concept': 'group_stage_entity',
  'cdm_profile': 'observation_simple',
  'notes':